# QoL Index Dashboard

`Nika Karimi Jashni, Tie Ma, Yunhan Liu` <br>

Last Updated: June 6, 2024

In [14]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import json
from dash.dependencies import Input, Output

# Data for the specified cities
data = {
    'City': ['Calgary', 'Edmonton', 'Hamilton', 'Kingston', 'Kitchener', 'London',
             'Montreal', 'Ottawa', 'Quebec City', 'Toronto', 'Vancouver', 'Winnipeg'],
    'Latitude': [51.0447, 53.5461, 43.2557, 44.2312, 43.4516, 42.9849, 
                 45.5017, 45.4215, 46.8139, 43.6511, 49.2827, 49.8951],
    'Longitude': [-114.0719, -113.4938, -79.8711, -76.4859, -80.4925, -81.2453, 
                  -73.5673, -75.6972, -71.2082, -79.3470, -123.1207, -97.1384],
    'Value1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'Value2': [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
    'Value3': [6, 5, 4, 3, 2, 1, 12, 11, 10, 9, 8, 7],
    'Value4': [7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6],
    'Value5': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
}
df = pd.DataFrame(data)

# Load GeoJSON data for Canada borders from local file
geojson_path = 'D:/Sata_Can_intership/canada.geojson'
with open(geojson_path) as f:
    geojson = json.load(f)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define a color-blind friendly color palette
color_palette = [
    "#377eb8",  # Blue
    "#ff7f00",  # Orange
    "#4daf4a",  # Green
    "#f781bf",  # Pink
    "#a65628",  # Brown
    "#984ea3",  # Purple
    "#999999",  # Grey
    "#e41a1c",  # Red
    "#dede00"   # Yellow
]

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Canada City Data Points", style={'textAlign': 'center', 'color': '#333333'}),
    html.Div([
        html.Div([
            html.H3("City Value", style={'color': '#333333'}),
            html.P("Changes as the city changes", style={'color': '#333333'}),
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top', 'backgroundColor': '#f7f7f7', 'padding': '10px', 'borderRadius': '5px', 'margin': '10px'}),

        html.Div([
            html.H3("Map", style={'color': '#333333'}),
            dcc.Graph(id='map', style={'height': '60vh'}),
            html.P("Highlight selected cities and Canada itself", style={'color': '#333333'}),
            html.P("Selected cities: Montreal, Vancouver, Winnipeg, Kingston, Kitchener, Quebec City, Ottawa, Toronto, Edmonton, Calgary, Hamilton", style={'color': '#333333'})
        ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top', 'backgroundColor': '#f7f7f7', 'padding': '10px', 'borderRadius': '5px', 'margin': '10px'}),

        html.Div([
            html.H3("Scores", style={'color': '#333333'}),
            html.Div([
                html.P("Environment", style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-1',
                    min=0,
                    max=5,
                    step=0.1,
                    value=0,
                    marks={i: str(i) for i in range(0, 6)}
                ),
                html.Div(id='score-1', style={'textAlign': 'center', 'color': '#333333'}),
            ]),
            html.Div([
                html.P("Society", style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-2',
                    min=0,
                    max=5,
                    step=0.1,
                    value=0,
                    marks={i: str(i) for i in range(0, 6)}
                ),
                html.Div(id='score-2', style={'textAlign': 'center', 'color': '#333333'}),
            ]),
            html.Div([
                html.P("Access", style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-3',
                    min=0,
                    max=5,
                    step=0.1,
                    value=0,
                    marks={i: str(i) for i in range(0, 6)}
                ),
                html.Div(id='score-3', style={'textAlign': 'center', 'color': '#333333'}),
            ]),
            html.Div([
                html.P("Prospects", style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-4',
                    min=0,
                    max=5,
                    step=0.1,
                    value=0,
                    marks={i: str(i) for i in range(0, 6)}
                ),
                html.Div(id='score-4', style={'textAlign': 'center', 'color': '#333333'}),
            ]),
            html.Div([
                html.P("Affordability", style={'color': '#333333'}),
                dcc.Slider(
                    id='slider-5',
                    min=0,
                    max=5,
                    step=0.1,
                    value=0,
                    marks={i: str(i) for i in range(0, 6)}
                ),
                html.Div(id='score-5', style={'textAlign': 'center', 'color': '#333333'}),
            ]),
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top', 'backgroundColor': '#f7f7f7', 'padding': '10px', 'borderRadius': '5px', 'margin': '10px'}),
    ], style={'display': 'flex', 'flex-direction': 'row', 'backgroundColor': '#eaeaea', 'padding': '20px', 'borderRadius': '10px'}),

    html.Div([
        html.H3("City Details", style={'color': '#333333'}),
        html.Table([
            html.Thead(
                html.Tr([html.Th(col) for col in ["City", "Environment", "Society", "Access", "Prospects", "Affordability"]])
            ),
            html.Tbody([
                html.Tr([
                    html.Td(city),
                    html.Td(id=f'env-{city}'),
                    html.Td(id=f'soc-{city}'),
                    html.Td(id=f'acc-{city}'),
                    html.Td(id=f'pro-{city}'),
                    html.Td(id=f'aff-{city}')
                ]) for city in data['City']
            ])
        ], style={'width': '100%', 'backgroundColor': '#ffffff', 'padding': '10px', 'borderRadius': '5px', 'margin': '10px'}),
    ], style={'backgroundColor': '#f7f7f7', 'padding': '20px', 'borderRadius': '10px', 'margin': '10px'}),
], style={'backgroundColor': '#d0e4f7', 'padding': '20px', 'borderRadius': '10px'})

@app.callback(
    [Output('map', 'figure'),
     Output('city-rank', 'children'),
     Output('score-1', 'children'),
     Output('score-2', 'children'),
     Output('score-3', 'children'),
     Output('score-4', 'children'),
     Output('score-5', 'children')] +
    [Output(f'env-{city}', 'children') for city in data['City']] +
    [Output(f'soc-{city}', 'children') for city in data['City']] +
    [Output(f'acc-{city}', 'children') for city in data['City']] +
    [Output(f'pro-{city}', 'children') for city in data['City']] +
    [Output(f'aff-{city}', 'children') for city in data['City']],
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value'),
     Input('map', 'clickData')]
)
def update_map(weight1, weight2, weight3, weight4, weight5, clickData):
    if clickData is None:
        selected_city = None
    else:
        selected_city = clickData['points'][0]['text']
    
    if selected_city:
        selected_row = df[df['City'] == selected_city].iloc[0]
        score1 = weight1 * selected_row['Value1']
        score2 = weight2 * selected_row['Value2']
        score3 = weight3 * selected_row['Value3']
        score4 = weight4 * selected_row['Value4']
        score5 = weight5 * selected_row['Value5']
    else:
        score1 = score2 = score3 = score4 = score5 = 0
    
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Value1'] + 
                         weight2 * df['Value2'] + 
                         weight3 * df['Value3'] + 
                         weight4 * df['Value4'] + 
                         weight5 * df['Value5'])
    
    # Add ranking
    df['Rank'] = df['WeightedAvg'].rank(ascending=False).astype(int)
    
    # Sort by rank
    df_sorted = df.sort_values(by='Rank')
    
    # Prepare city rank display
    city_rank = "\n".join([f"{rank}. {city}" for rank, city in zip(df_sorted['Rank'], df_sorted['City'])])
    
    # Create the map
    fig = px.scatter_mapbox(
        df, lat="Latitude", lon="Longitude", size="WeightedAvg",
        hover_name="City", text="City",
        mapbox_style="carto-positron", zoom=3, center={"lat": 56.1304, "lon": -106.3468},
        height=600, width=1000
    )

    fig.update_traces(marker=dict(color="red"), selector=dict(type="scattermapbox"))

    fig.update_layout(
        mapbox=dict(
            layers=[
                {
                    "source": geojson,
                    "type": "line",
                    "color": "blue",
                    "line": {"width": 2}
                }
            ]
        ),
        margin={"r":0,"t":0,"l":0,"b":0},
        paper_bgcolor='#eaeaea'
    )
    
    # Prepare detailed city scores
    env_scores = [f"{weight1 * row['Value1']:.2f}" for _, row in df.iterrows()]
    soc_scores = [f"{weight2 * row['Value2']:.2f}" for _, row in df.iterrows()]
    acc_scores = [f"{weight3 * row['Value3']:.2f}" for _, row in df.iterrows()]
    pro_scores = [f"{weight4 * row['Value4']:.2f}" for _, row in df.iterrows()]
    aff_scores = [f"{weight5 * row['Value5']:.2f}" for _, row in df.iterrows()]
    
    return [fig, city_rank, f"Score: {score1}", f"Score: {score2}", f"Score: {score3}", f"Score: {score4}", f"Score: {score5}"] + env_scores + soc_scores + acc_scores + pro_scores + aff_scores

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # You can choose any available port, e.g., 8051


In [20]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Sample data: Replace this with your actual data
data = {
    'City': ['Montreal', 'Vancouver', 'Winnipeg', 'Kingston', 'Kitchener', 'Quebec City', 'Ottawa', 'Toronto', 'Edmonton', 'Calgary', 'Hamilton'],
    'Latitude': [45.5, 49.2, 49.9, 44.2, 43.4, 46.8, 45.4, 43.7, 53.5, 51.0, 43.2],
    'Longitude': [-73.6, -123.1, -97.1, -76.5, -80.5, -71.2, -75.7, -79.4, -113.5, -114.1, -79.9],
    'Value1': [1, 2, 3, 4, 5, 2, 3, 4, 5, 3, 4],
    'Value2': [5, 4, 3, 2, 1, 3, 4, 5, 2, 1, 5],
    'Value3': [2, 3, 4, 1, 5, 4, 1, 5, 3, 2, 1],
    'Value4': [3, 1, 2, 5, 4, 5, 1, 3, 4, 2, 3],
    'Value5': [4, 5, 1, 3, 2, 1, 3, 2, 5, 4, 2]
}
df = pd.DataFrame(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.Div([
        html.H2("City value"),
        html.Ul([
            html.Li("Changes as the city changes")
        ])
    ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    html.Div([
        html.H1("Map"),
        html.P("Highlight selected cities and Canada itself"),
        html.P("Selected cities: Montreal, Vancouver, Winnipeg, Kingston, Kitchener, Quebec City, Ottawa, Toronto, Edmonton, Calgary, Hamilton"),
        dcc.Graph(id='map')
    ], style={'width': '60%', 'display': 'inline-block', 'textAlign': 'center'}),
    
    html.Div([
        html.H3("Scores"),
        html.Div([
            html.P("Environment"),
            dcc.Slider(
                id='slider-1',
                min=0,
                max=1,
                step=0.1,
                value=0.2,
                marks={i/10: str(i/10) for i in range(0, 11)}
            )
        ]),
        html.Div([
            html.P("Society"),
            dcc.Slider(
                id='slider-2',
                min=0,
                max=1,
                step=0.1,
                value=0.2,
                marks={i/10: str(i/10) for i in range(0, 11)}
            )
        ]),
        html.Div([
            html.P("Access"),
            dcc.Slider(
                id='slider-3',
                min=0,
                max=1,
                step=0.1,
                value=0.2,
                marks={i/10: str(i/10) for i in range(0, 11)}
            )
        ]),
        html.Div([
            html.P("Prospects"),
            dcc.Slider(
                id='slider-4',
                min=0,
                max=1,
                step=0.1,
                value=0.2,
                marks={i/10: str(i/10) for i in range(0, 11)}
            )
        ]),
        html.Div([
            html.P("Affordability"),
            dcc.Slider(
                id='slider-5',
                min=0,
                max=1,
                step=0.1,
                value=0.2,
                marks={i/10: str(i/10) for i in range(0, 11)}
            )
        ])
    ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    html.Div([
        html.Table([
            html.Tr([
                html.Th("City"),
                html.Th("Environment"),
                html.Th("Society"),
                html.Th("Access"),
                html.Th("Prospect"),
                html.Th("Affordability")
            ])] + [
            html.Tr([
                html.Td(city),
                html.Td(dcc.Slider(min=0, max=1, step=0.1, value=0.2)),
                html.Td(dcc.Slider(min=0, max=1, step=0.1, value=0.2)),
                html.Td(dcc.Slider(min=0, max=1, step=0.1, value=0.2)),
                html.Td(dcc.Slider(min=0, max=1, step=0.1, value=0.2)),
                html.Td(dcc.Slider(min=0, max=1, step=0.1, value=0.2))
            ]) for city in df['City']
        ])
    ], style={'width': '100%', 'display': 'inline-block', 'textAlign': 'center'})
])

@app.callback(
    Output('map', 'figure'),
    [Input('slider-1', 'value'),
     Input('slider-2', 'value'),
     Input('slider-3', 'value'),
     Input('slider-4', 'value'),
     Input('slider-5', 'value')]
)
def update_map(weight1, weight2, weight3, weight4, weight5):
    # Calculate weighted average
    df['WeightedAvg'] = (weight1 * df['Value1'] + 
                         weight2 * df['Value2'] + 
                         weight3 * df['Value3'] + 
                         weight4 * df['Value4'] + 
                         weight5 * df['Value5'])

    # Create the map
    fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", size="WeightedAvg", 
                            hover_name="City", 
                            mapbox_style="carto-positron", 
                            size_max=15, zoom=3)
    
    return fig

# Run the app
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)  # Change port number if 8050 is already in use

